# 5.9 - XGBoost

In [28]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score,precision_score,recall_score
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from IPython.display import display
from sklearn.preprocessing import scale
from keras.wrappers.scikit_learn import KerasClassifier

import seaborn as sns

from keras.datasets import mnist
from keras.layers import *
from keras.models import *
from time import time
from keras.utils import np_utils

from xgboost import XGBClassifier

os.chdir('D:\\Users\\Diogo\\Documents\\Pós Graduação\\Módulos\\TCC\\DataSets\\Futebol Brasileiro')

In [74]:
#Carregando os dados do Data Frame

df_dados=pd.read_csv('dados_normalizados.csv',sep=',')

df_dados.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,ID,campeonato,rodata,data,clube_mandante,clube_visitante,qtd_cartao_amarelo_mandante,...,FalAma_mandante,FalVer_mandante,FalAma_visitante,FalVer_visitante,FalAmaTemp_mandante,FalVerTemp_mandante,FalAmaTemp_visitante,FalVerTemp_visitante,Target_mandante_amarelo,Target_visitante_amarelo
0,0,0,0,4987,2015,1,9/5/2015,Chapecoense,Coritiba,0.400000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,1,1,1,4988,2015,1,9/5/2015,Palmeiras,Atletico-MG,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1
2,2,2,2,4989,2015,1,9/5/2015,Fluminense,Joinville,0.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
3,3,3,3,4990,2015,1,10/5/2015,Gremio,Ponte Preta,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
4,4,4,4,4991,2015,1,10/5/2015,Athletico-PR,Internacional,0.200000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [75]:
df_dados_norm = df_dados.copy(deep=True)

In [76]:
#Selecionando as colunas que serão utilizadas no modelo

df_dados_norm_visitante=df_dados_norm[['classico_regional','MedAma5_mandante','MedAma5_visitante',
'MedAmaTemp_mandante','MedAmaTemp_visitante','MedAmaH_mandante',
'MedAmaH_visitante','MaxAmaTemp_mandante','MinAmaTemp_mandante',
'MaxAmaTemp_visitante','MinAmaTemp_visitante','ScoreMed_mandante','ScoreLevel_mandante','ScoreMedTemp_mandante',
                             'ScoreLevelTemp_mandante','FalAma_mandante','FalAmaTemp_mandante','FalAma_visitante',
                             'FalAmaTemp_visitante','ScoreMed_visitante','ScoreLevel_visitante',
                            'ScoreMedTemp_visitante']]

target= df_dados_norm[['Target_visitante_amarelo']]
y=target.copy()
X=df_dados_norm_visitante.copy()

In [77]:
#Dividindo os Dataframes que serão utilzados no modelo

X_train=df_dados_norm_visitante[:2128]
X_test=df_dados_norm_visitante[2128:3040]
y_train=target[:2128]
y_test=target[2128:3040]

In [14]:
X_train.columns

Index(['classico_regional', 'MedAma5_mandante', 'MedAma5_visitante',
       'MedAmaTemp_mandante', 'MedAmaTemp_visitante', 'MedAmaH_mandante',
       'MedAmaH_visitante', 'MaxAmaTemp_mandante', 'MinAmaTemp_mandante',
       'MaxAmaTemp_visitante', 'MinAmaTemp_visitante', 'ScoreMed_mandante',
       'ScoreLevel_mandante', 'ScoreMedTemp_mandante',
       'ScoreLevelTemp_mandante', 'FalAma_mandante', 'FalAmaTemp_mandante',
       'FalAma_visitante', 'FalAmaTemp_visitante', 'ScoreMed_visitante',
       'ScoreLevel_visitante', 'ScoreLevelTemp_mandante',
       'ScoreMedTemp_visitante'],
      dtype='object')

In [33]:

#Código de para aplicar o Grid Search

model = XGBClassifier()

# Defina os parâmetros que deseja otimizar
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight':[0.1,0.2,0.5,1]
}

# Crie o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

# Realize a pesquisa de grade no conjunto de treinamento
grid_search.fit(X_train, y_train)

# Obtenha os melhores hiperparâmetros
best_params = grid_search.best_params_
print('Melhores Hiperparâmetros:', best_params)

# Faça predições no conjunto de teste usando os melhores hiperparâmetros
y_pred = grid_search.predict(X_test)

# Avalie a acurácia do modelo usando os melhores hiperparâmetros
accuracy = accuracy_score(y_test, y_pred)
print('Acurácia com Melhores Hiperparâmetros: {:.2f}%'.format(accuracy * 100))

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[11:56:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning

[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning

[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[11:56:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:04:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:04:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:04:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Melhores Hiperparâmetros: {'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 50, 'scale_pos_weight': 1}
Acurácia com Melhores Hiperparâmetros: 62.57%


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [78]:
#Código para treinamento do modelo e exibição dos resultados

modelo = XGBClassifier(learning_rate=0.01, max_depth= 3, min_child_weight= 5, n_estimators= 50, scale_pos_weight= 1)
modelo.fit(X_train, y_train)

pred = modelo.predict(X_test)
y_pred = (pred > 0.5).astype(int)

xpred = modelo.predict(X)
x_pred= (xpred > 0.5).astype(int)

xgb = accuracy_score(y_test,y_pred)

print ('\n Matriz de Confusão')
print(confusion_matrix(y_test,y_pred))
print('\n Acurácia: %.2f%%' % (xgb * 100.0))
print('\n Relatório Completo')
print(classification_report(y,x_pred))
print('\n Relatório Teste')
print(classification_report(y_test,y_pred))

c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[21:19:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Matriz de Confusão
[[457  60]
 [281 113]]

 Acurácia: 62.57%

 Relatório Completo
              precision    recall  f1-score   support

           0       0.59      0.89      0.71      1655
           1       0.67      0.27      0.39      1384

    accuracy                           0.61      3039
   macro avg       0.63      0.58      0.55      3039
weighted avg       0.63      0.61      0.56      3039


 Relatório Teste
              precision    recall  f1-score   support

           0       0.62      0.88      0.73       517
           1       0.65      0.29      0.40       394

    accuracy                           0.63       911
   macro avg       0.64      0.59      0.56

In [80]:
#Inserção dos resultados no Dataframe para análise. Esses são os resultados finais das performances dos modelos.

df_resultados_visitantes=pd.read_csv('df_resultados_visitantes.csv',sep=',')
dados = [
    {"tipo": "Visitantes", "modelo": "XGBoost", "acuracia": xgb.round(4), "precisao_teste": precision_score(y_test,y_pred,average='macro').round(4), "precisao_geral": precision_score(y,x_pred,average='macro').round(4), "recall_teste": recall_score(y_test,y_pred,average='macro').round(4), "recall_geral": recall_score(y,x_pred,average='macro').round(4)}
]
df_resultados_visitantes = df_resultados_visitantes.append(dados, ignore_index=True)
df_resultados_visitantes

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,tipo,modelo,acuracia,precisao_teste,precisao_geral,recall_teste,recall_geral
0,0.0,0.0,0.0,Visitantes,Regressão Logística,0.5631,0.5330,0.5654,0.5191,0.5332
1,1.0,1.0,NaN,Visitantes,SVC,0.5686,0.5621,0.6357,0.5025,0.5108
2,2.0,NaN,NaN,Visitantes,Redes Neurais,0.5467,0.5331,0.5989,0.5320,0.5912
3,NaN,NaN,NaN,Visitantes,XGBoost,0.6257,0.6362,0.6300,0.5854,0.5799


In [81]:
df_dados_norm = df_dados.copy(deep=True)

In [82]:
#Selecionando as colunas que serão utilizadas no modelo

df_dados_norm_mandante=df_dados_norm[['classico_regional','MedAma5_mandante','MedAma5_visitante',
'MedAmaTemp_mandante','MedAmaTemp_visitante','MedAmaH_mandante',
'MedAmaH_visitante','MaxAmaTemp_mandante','MinAmaTemp_mandante',
'MaxAmaTemp_visitante','MinAmaTemp_visitante','ScoreMed_mandante','ScoreLevel_mandante','ScoreMedTemp_mandante',
                             'ScoreLevelTemp_mandante','FalAma_mandante','FalAmaTemp_mandante','FalAma_visitante',
                             'FalAmaTemp_visitante','ScoreMed_visitante','ScoreLevel_visitante',
                            'ScoreMedTemp_visitante']]

target= df_dados_norm[['Target_mandante_amarelo']]
y=target.copy()
X=df_dados_norm_mandante.copy()

In [83]:
#Dividindo os Dataframes que serão utilzados no modelo

X_train=df_dados_norm_mandante[:2128]
X_test=df_dados_norm_mandante[2128:3040]
y_train=target[:2128]
y_test=target[2128:3040]

In [72]:
#Código de para aplicar o Grid Search

model = XGBClassifier()

# Defina os parâmetros que deseja otimizar
param_grid = {
    'max_depth': [3, 5, 7, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'min_child_weight': [1, 3, 5],
    'scale_pos_weight':[0.1,0.2,0.5,1]
}

# Crie o objeto GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)

# Realize a pesquisa de grade no conjunto de treinamento
grid_search.fit(X_train, y_train)

# Obtenha os melhores hiperparâmetros
best_params = grid_search.best_params_
print('Melhores Hiperparâmetros:', best_params)

# Faça predições no conjunto de teste usando os melhores hiperparâmetros
y_pred = grid_search.predict(X_test)

# Avalie a acurácia do modelo usando os melhores hiperparâmetros
accuracy = accuracy_score(y_test, y_pred)
print('Acurácia com Melhores Hiperparâmetros: {:.2f}%'.format(accuracy * 100))

Fitting 3 folds for each of 432 candidates, totalling 1296 fits
[12:32:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:32:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:32:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning

[12:32:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:32:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

[12:32:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:32:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[12:39:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:39:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[12:39:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Melhores Hiperparâmetros: {'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 100, 'scale_pos_weight': 0.5}
Acurácia com Melhores Hiperparâmetros: 62.68%


In [84]:
#Código para treinamento do modelo e exibição dos resultados

modelo = XGBClassifier(learning_rate= 0.1, max_depth=3, min_child_weight=3,n_estimators=100,scale_pos_weight= 0.5)
modelo.fit(X_train, y_train)

pred = modelo.predict(X_test)
y_pred = (pred > 0.5).astype(int)

xpred = modelo.predict(X)
x_pred= (xpred > 0.5).astype(int)

xgb = accuracy_score(y_test,y_pred)

print ('\n Matriz de Confusão')
print(confusion_matrix(y_test,y_pred))
print('\n Acurácia: %.2f%%' % (xgb * 100.0))
print('\n Relatório Completo')
print(classification_report(y,x_pred))
print('\n Relatório Teste')
print(classification_report(y_test,y_pred))

c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
c:\users\diogo\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[21:22:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.

 Matriz de Confusão
[[543  10]
 [330  28]]

 Acurácia: 62.68%

 Relatório Completo
              precision    recall  f1-score   support

           0       0.64      0.99      0.78      1877
           1       0.92      0.11      0.20      1162

    accuracy                           0.66      3039
   macro avg       0.78      0.55      0.49      3039
weighted avg       0.75      0.66      0.56      3039


 Relatório Teste
              precision    recall  f1-score   support

           0       0.62      0.98      0.76       553
           1       0.74      0.08      0.14       358

    accuracy                           0.63       911
   macro avg       0.68      0.53      0.45

In [86]:
#Inserção dos resultados no Dataframe para análise.Esses são os resultados finais das performances dos modelos.

df_resultados_mandantes=pd.read_csv('df_resultados_mandantes.csv',sep=',')
dados = [
    {"tipo": "Mandantes", "modelo": "XGBoost", "acuracia": xgb.round(4), "precisao_teste": precision_score(y_test,y_pred,average='macro').round(4), "precisao_geral": precision_score(y,x_pred,average='macro').round(4), "recall_teste": recall_score(y_test,y_pred,average='macro').round(4), "recall_geral": recall_score(y,x_pred,average='macro').round(4)}

    ]
df_resultados_mandantes = df_resultados_mandantes.append(dados, ignore_index=True)
df_resultados_mandantes

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,tipo,modelo,acuracia,precisao_teste,precisao_geral,recall_teste,recall_geral
0,0.0,0.0,0.0,Mandantes,Regressão Logística,0.6103,0.5646,0.5929,0.5384,0.5250
1,1.0,1.0,NaN,Mandantes,SVC,0.6037,0.5396,0.6832,0.5081,0.5439
2,2.0,NaN,NaN,Mandantes,Redes Neurais,0.6158,0.5800,0.6544,0.5456,0.5796
3,NaN,NaN,NaN,Mandantes,XGBoost,0.6268,0.6794,0.7800,0.5301,0.5532
